In [4]:
import matplotlib.pyplot as plt
import numpy as np
import astropy.io.fits as pf
from glob import glob
from astropy import units as u
import pymc3 as pm
from pymc3 import Model, Normal, Gamma, Uniform
import theano.tensor as tt
from pymc3 import NUTS, Metropolis, sample
from pymc3 import find_MAP
from scipy import optimize
from astropy.cosmology import WMAP9 as cosmo
import os
from scipy.integrate import quad
from astropy.convolution import convolve
import time
from scipy.stats import norm
from pymc3.math import erf
import scipy

import warnings
warnings.filterwarnings("ignore")

In [5]:
zs={"UGC12158":0.031,"CGCG023-005":0.03334,"CGCG048-051":0.027754,"NGC5185":0.024560,
  "UGC9530":.02854,"E472-G007":.03032,"CGCG063-098":.02992,"NGC2370":.01835,"E287-G40":.03087,
   "ASASSN-15fr":.033436,"LSQ12gdj":.030,"CGCG048-099":0.032,"ASASSN-14hr":0.03362,"ASASSN-14ad":.0264,
   "E154-G010":.0186,"E153-G020":0.0197,"LSQ13ry":.0299,"NGC7580":.0148,'id11':0.05574 }

Avs={"UGC12158":0.151,"CGCG023-005":0.262,"CGCG048-051":0.124,"NGC5185":0.067,"UGC9530":.077,
    "E472-G007":0.064,"CGCG063-098":0.134,"NGC2370":0.166,"E287-G40":.078,
    "ASASSN-15fr":.104,"LSQ12gdj":0.145,"CGCG048-099":.143,"ASASSN-14hr":.041,"ASASSN-14ad":.050,"E154-G010":.082,
    "E153-G020":.073,"LSQ13ry":0.130,"NGC7580":.125,'id11':.301}

CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00','deeppink','k']

gals=["E287-G40","CGCG023-005","CGCG048-099","NGC5185","NGC2370","UGC9530","ASASSN-15fr","ASASSN-14hr",
      "LSQ13ry","E153-G020"]


In [8]:
def get_sigma_i_kinematics(w):
    #input wavelength in angstroms and the instrumental resolution in km/s from the kinematics paper measurements
    a,b,c=np.loadtxt("muse_res_kinematics.txt")
    fwhm=(a * w**2) + (b*w) + c #Angstroms
    sig=fwhm/(2*np.sqrt(2*np.log(2))) #Angstroms
    sigma_i = sig * (3e5) / w #km/s
    
    return sigma_i #km/s

def get_sigma_i_calibration(w):
    #input wavlength in angstroms and get the instrumental resoluiton in km/s from the LMS calibration file
    a,b,c,d=np.loadtxt("muse_res_calibration.txt")
    fwhm=(a*w**3)+(b*w**2)+(c*w)+d #Angstroms
    sig=fwhm/(2*np.sqrt(2*np.log(2))) #Angstroms
    sigma_i=sig*(3e5) / w #km/s
    
    return sigma_i #km/s

def get_sigma_i_manual(w):
    #input wavelength in angstroms and get the manual + header shifted instrumental resolution in km/s
    m,b=np.loadtxt("muse_res_manual.txt")
    
    R=m*(w/10) + b
    
    delta=236.07 #shift from header value
    
    R+=delta
    
    sigma_i=3e5/R/(2*np.sqrt(2*np.log(2))) #km/s

    return sigma_i #km/s

In [9]:
def likelihood(x,logL,logdisp,av,v_r,FOIII,c,line,gal,resolution,plot_version=False):
    z=zs[gal] #get galaxy's redshift
    DL=float(cosmo.luminosity_distance(z).to(u.cm)/u.cm) #get galaxy's distance
    lines={"HB":4861.35,"OIII":5006.84,"HA":6562.79} #air wavelengths in angstroms of emission lines
    
    #intrinsic luminosity and velocity dispersion
    L=10**logL #Luminosity in erg/s
    disp=10**logdisp #intrinsic velocity dispersion in km/s
    
    #extinction corrected flux of Hbeta and Halpha
    FHB_corr=L/(4*np.pi*DL**2) #extinction corrected flux of Hbeta in erg/s/cm2
    FHA_corr=FHB_corr*2.86 #extinction corrected flux of Halpha in erg/s/cm2 (assuming case B)
    
    #extinction correction parameters from Gordon extinction law
    Rv=2.77 #Rv from Gordon extinction law
    kHB=3.33 #kappa from Gordon extinction law
    kHA=2.22
    
    #"Observed"/reddened fluxes of Hbeta and Halpha
    FHB=FHB_corr/(10**(.4*av*kHB/Rv)) #observed flux of Hbeta in erg/s/cm2
    FHA=FHA_corr/(10**(.4*av*kHA/Rv)) #observed flux of Halpha in erg/s/cm2
    
    #Observed emission line center given galaxy redshift (z) + line of sight velocity (v_r)
    x0=(1+z)*lines[line]*(1+(v_r/3e5)) #Angstroms
    
    #get instrumental resolution at line center from given method
    sigma_i=0 #km/s
    if resolution=='calibration': #getting sigma_i from LMS calibration file
        sigma_i+=get_sigma_i_calibration(x0)
    elif resolution=="kinematics": #getting sigma_i from Kinematics paper
        sigma_i+=get_sigma_i_kinematics(x0)
    elif resolution=="manual": #gettting sigma_i from manual + header shift
        sigma_i+=get_sigma_i_manual(x0)
    
    #get fine structure and temperature broadening for specific line, specify flux
    simga_fs=0 #km/s
    sigma_T=0 #km/s
    F=0 #erg/s/cm2
    
    if line=="HA":
        sigma_fs+=np.sqrt(10.233)
        sigma_T+=9.1#km/s
        F+=FHA
    elif line=="HB":
        sigma_fs+=np.sqrt(5.767)
        sigma_T+=9.1
        F+=FHB
    elif line=="OIII":
        sigma_fs+=0
        sigma_T+=2.3
        F+=FOIII
    
    #get observed sigma in Angstroms by adding all components in quadrature
    sig=np.sqrt(disp**2 + sigma_i**2 + sigma_fs**2+sigma_T**2)*x0/3e5
    
    #get normalization of gaussian profile
    A=F/(np.sqrt(2*np.pi))/np.abs(sig)
    
    #perform integration to get average flux density over pixel width (1.25 Angstroms for MUSE)
    width=1.25
    left=x-(1.25/2.)
    right=x+(1.25/2.)
    
    if plot_version ==False:
        #version for fitting process which uses pymc3's erf
        integrated=A*sig*np.sqrt(np.pi/2) *(erf((x0-left)/(np.sqrt(2)*sig)) - erf((x0-right)/(np.sqrt(2)*sig)))
    elif plot_version ==True:
        #version for plotting/simple calculate which uses scipy's erf
        integrated=A*sig*np.sqrt(np.pi/2) *(scipy.special.erf((x0-left)/(np.sqrt(2)*sig)) - scipy.special.erf((x0-right)/(np.sqrt(2)*sig)))
    
    #divide by pixel width to get flux density and then add continuum value
    integrated_flux=integrated/width + c #flux density in erg/s/cm2/A

    return integrated_flux #erg/s/cm2/A

In [ ]:
def mcmc_fit(gal,hid,resolution):
    z=zs[gal] #galaxy redshift
    
    mu=8 #assumed magnification = 8
    wav,spec,noise=np.loadtxt(gal+"/spectra"+"_"+str(mu)+"/"+str(hid)+"_spectrum.txt")
    
    size=12 #extraction window width in pixels (total width = size *2)
    
    #get H-beta wavelength
    HB_rest=4861.35
    HB_obs=HB_rest*(1+z)
    HB_obs_ind=np.argmin(np.abs(HB_obs-wav))
    HBleft=HB_obs_ind-size
    HBright=HB_obs_ind+size
    
    xHB=wav[HBleft:HBright+1]
    yHB=spec[HBleft:HBright+1]
    nHB=noise[HBleft:HBright+1]
    
    #get H-alpha wavelength
    HA_rest=6562.79
    HA_obs=HA_rest*(1+z)
    HA_obs_ind=np.argmin(np.abs(HA_obs-wav))
    HAleft=HA_obs_ind-size
    HAright=HA_obs_ind+size
    
    xHA=wav[HAleft:HAright+1]
    yHA=spec[HAleft:HAright+1]
    nHA=noise[HAleft:HAright+1]
    
    #get OIII wavelength
    OIII_rest=5006.84
    OIII_obs=OIII_rest*(1+z)
    OIII_obs_ind=np.argmin(np.abs(OIII_obs-wav))
    OIIIleft=OIII_obs_ind-size
    OIIIright=OIII_obs_ind+size
    
    xOIII=wav[OIIIleft:OIIIright+1]
    yOIII=spec[OIIIleft:OIIIright+1]
    nOIII=noise[OIIIleft:OIIIright+1]
    
    #start mcmc fitting
    gaussian_model=Model()
    
    with gaussian_model:
        #define priors (using Uniform priors)

        #prior for radial velocity in km/s
        v_r=Uniform('v_r',lower=-200,upper=200,shape=1)

        #prior for log10 (velocity dispersion) in log(km/s)
        logdisp=Uniform('logdisp',lower=0,upper=3,shape=1) 

        #prior for background offset
        #Uniform priors from -1e-15 to 1e-15 erg/s/cm2/A
        cHB=Uniform('cHB',lower=-1e-15,upper=1e-15,shape=1)
        cHA=Uniform('cHA',lower=-1e-15,upper=1e-15,shape=1)
        cOIII=Uniform('cOIII',lower=-1e-15,upper=1e-15,shape=1)

        #prior for log10 (HB luminosity) in log(erg/s)
        logL=Uniform("logL",lower=37,upper=43,shape=1) 
        
        #prior for extinction in magnitudes
        av=Uniform("av",lower=0,upper=10,shape=1)
        
        #prior for observed OIII flux in erg/s/cm2
        FOIII=Uniform('FOIII',lower=-10e-15,upper=10e-15,shape=1)
    
        # Expected value from likelihood function
        muHB = likelihood(xHB,logL,logdisp,av,v_r,FOIII,cHB,"HB",gal,resolution,plot_version=False)
        muOIII=likelihood(xOIII,logL,logdisp,av,v_r,FOIII,cOIII,'OIII',gal,resolution,plot_version=False)
        muHA=likelihood(xHA,logL,logdisp,av,v_r,FOIII,cHA,"HA",gal,resolution,plot_version=False)

        #observed value
        Y_obs_HB = Normal('Y_obs_HB', mu=muHB, tau=1/(nHB)**2, observed=yHB)
        Y_obs_OIII=Normal('Y_obs_OIII',mu=muOIII,tau=1/(nOIII)**2,observed=yOIII)
        Y_obs_HA=Normal('Y_obs_HA',mu=muHA,tau=1/(nHA)**2,observed=yHA)
        
        #sampling
        trace = pm.sample(10000,chains=1,tune=2000,target_accept=.98)
        
        if len(glob(gal+"/traces*"))==0:
            os.mkdir(gal+"/traces")
        
        np.savetxt(gal+"/traces/tr_"+str(hid)+".txt",trace,fmt="%s")

    return trace